In [1]:
# коллабные импорты
from google.colab import drive
from google.colab import files

In [2]:
!mkdir GD

In [3]:
drive.mount('/GD',force_remount=True)

Mounted at /GD


In [4]:
!cp -r /GD/'My Drive'/cars.zip .

In [ ]:
!unzip cars.zip

In [9]:
!ls cars


01  02	03  Avar  Medical  Nothing  Post  Tractor  Truck


In [25]:
!ls train

01  02	03  Avar  Medical  Nothing  Post  Tractor  Truck


In [19]:
!mkdir longbox/Truck

In [24]:
!cp -r longbox/ train/

In [26]:
import os, shutil
import random as rn

In [30]:
# каталоги источники
train_dirs = ['cars/01', 'cars/02', 'cars/03', 'cars/Avar', 'cars/Medical',  'cars/Nothing', 'cars/Post',
              'cars/Tractor', 'cars/Truck']

# каталоги приемники
main_dir = 'test'
test_dirs = [main_dir+'/01', main_dir+'/02', main_dir+'/03', main_dir+'/Avar', main_dir+'/Medical',  main_dir+'/Nothing',
              main_dir+ '/Post',
              main_dir+'/Tractor', main_dir+'/Truck']

In [ ]:
for i in range(len(train_dirs)):
    fnames = os.listdir(train_dirs[i])
    toCopy = len(fnames)//7

    filesToCopy = []
    for j in range(toCopy):
        filesToCopy.append(rn.choice(fnames))
    
    # cOPY
         
    for fi in filesToCopy:
        src = os.path.join(train_dirs[i],fi)
        dst = os.path.join(test_dirs[i], fi)
        shutil.copyfile(src,dst)

    
    for fi in filesToCopy:
        try:
            src = os.path.join(train_dirs[i],fi)
            os.remove(src)
            print(src+' deleted')
            #time.sleep(1)
        except:
            pass

In [35]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras import Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#import os
import pandas as pd
import math
import numpy as np
import sys
import matplotlib.pyplot as plt

import pandas as pd
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2

# Дебаты про выбор размера картинки для подачи на вход нейросети

Y = 126
X = 224

In [53]:
inputs = Input(shape = (Y, X, 3))

conv1_1 = layers.Conv2D(16, (3, 3), activation='relu' )(inputs)
conv1_2 = layers.Conv2D(16, (3, 3), activation='relu')(conv1_1)
pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1_2)

conv2_1 = layers.Conv2D(32, (3, 3), activation='relu')(pool1)
conv2_2 = layers.Conv2D(32, (3, 3), activation='relu')(conv2_1)
pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2_2)

conv3_1 = layers.Conv2D(64, (3, 3), activation='relu')(pool2)
conv3_2 = layers.Conv2D(64, (3, 3), activation='relu')(conv3_1)
pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3_2)

flat = layers.Flatten()(pool3)

drop = layers.Dropout(0.5) (flat)

result = layers.Dense(9, activation = 'softmax') (drop)

model = models.Model(inputs=inputs, outputs=result)

model.compile(loss = 'categorical_crossentropy',
             optimizer=optimizers.Adam(lr=1e-3), metrics = ['acc'])


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

In [54]:
#train_datagen = ImageDataGenerator(rescale = 1./255)

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 25,
                                   width_shift_range = 0.15,
                                   height_shift_range = 0.15,
                                   shear_range = 0.15,
                                   zoom_range = 0.3,
                                   fill_mode = 'nearest') 

train_generator = train_datagen.flow_from_directory('cars',
                                                   target_size = (Y, X),
                                                   batch_size = 64,
                                                   class_mode = 'categorical')

test_datagen = ImageDataGenerator(rescale = 1./255)
                            
test_generator = test_datagen.flow_from_directory('longbox',
                                                   target_size = (Y, X),
                                                   batch_size = 64,
                                                   class_mode = 'categorical')


Found 14031 images belonging to 9 classes.
Found 2484 images belonging to 9 classes.


In [55]:
from tensorflow.keras.callbacks import  ModelCheckpoint

filepath = "car_net_simple_aug.h5"
chp = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')

In [ ]:
history = model.fit_generator(train_generator,
                         steps_per_epoch = 14031//64,
                         epochs = 5,
                         callbacks=[chp],
                         validation_data = test_generator,
                         validation_steps = 2484//64)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
 46/219 [=====>........................] - ETA: 1:13 - loss: 1.0309 - acc: 0.6332

In [57]:
!cp car_net_simple.h5 /GD/'My Drive'/car_net_simple.h5
!cp car_net_simple_aug.h5 /GD/'My Drive'/car_net_simple_aug.h5

In [44]:
train_generator.class_indices

{'01': 0,
 '02': 1,
 '03': 2,
 'Avar': 3,
 'Medical': 4,
 'Nothing': 5,
 'Post': 6,
 'Tractor': 7,
 'Truck': 8}

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras import Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#import os
import pandas as pd
import math
import numpy as np
import sys
import matplotlib.pyplot as plt

import pandas as pd
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2
